## 🛠️ Data Scrapping Jupyter Notebbok

In [1]:
from bs4 import BeautifulSoup as bs 
import requests
from ultralytics import YOLO
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from itertools import zip_longest
import cv2

#### Function for detection the damage

It is using my custom made yolov8 model


In [2]:
# function which is returning the number of object detected 
def number_object_detected(image):
    m = YOLO('yolov8m.pt')
    custom_model = YOLO('D:/COMSATS/FYP/Coding Part/ModelMaking/Computer_Vision_Part/3rd Try/runs/detect/train9/weights/best.pt') # custome yolo model path
    results = custom_model(image,verbose= False)

    dic = results[0].names
    classes = results[0].boxes.cls.cpu().numpy()
    probability = results[0].boxes.conf
    
    class_count = {}
    unique_elements, counts = np.unique(classes, return_counts=True)
    for e , count in zip(unique_elements,counts):
        a = dic[e]
        class_count[a] = count
    """    for r in results:
        im_array = r.plot(pil = True)  # plot a BGR numpy array of predictions
        array = im_array[..., ::-1]  # Convert BGR to RGB PIL image
        plt.axis("off")
        plt.imshow(array)
        plt.show()
    """


    return class_count 

## New function to detect the car 
This function is just using the simple yolov8m model to detect the car, and crop the image according to the co-ordinates and then pass the crop image to the damage detect function

In [3]:
def car_detection_and_Cropping(image_path):
    simple_yolo = YOLO('yolov8m.pt')
    r = simple_yolo(image_path,verbose = False)
    """
    for i in r:
        im_array = i.plot(pil=True)  # plot a BGR numpy array of predictions
        array = im_array[..., ::-1]  # Convert BGR to RGB PIL image
        plt.axis("off")
        plt.imshow(array)
        plt.show()
        """

    names = r[0].names
    boxes = r[0].boxes.xyxy.cpu().numpy().astype(int)
    classes = set(r[0].boxes.cls.cpu().numpy())
    classes2 =  [names[i] for i in classes]

    # checking if the detected object is the car or not 
    # if it is car then crop if not then pass the image as it is
    if boxes.size != 0 and 'car' in classes2:
      
      area = []
      for x1, y1, x2, y2 in boxes:
          area.append((x2 - x1) * (y2 - y1))
      max_index, max_a = max(enumerate(area), key=lambda x: x[1])
      

      # Load the image using OpenCV
      image = cv2.imread(image_path)
      
      
      # Crop the image
      crop_image = image[boxes[max_index][1]:boxes[max_index][3], boxes[max_index][0]:boxes[max_index][2]]

      # passing the crop image to the detection model

      class_c = number_object_detected(crop_image)
    else:
      class_c = number_object_detected(image_path)
    return class_c


Function for getting the body Condtion 

In [4]:
severity_points = {
    'scratch': 1,
    'dent': 2,
    'rust': 2,
    'paint-damage': 2,
    'crack':2
}

def calculate_condition_score(detections):
    total_score = 0
    for detection, count in detections.items():
        if detection in severity_points:
            total_score += severity_points[detection] * count
    return total_score

def normalize_score(score, max_score):
    return (score / max_score) * 10

def estimate_condition(detections):
    print("Detedtion list",detections)
    max_possible_score = sum(severity_points.values())  # Assuming all types of damage detected
    score = calculate_condition_score(detections)
    normalized_score = normalize_score(score, max_possible_score)
    
    # Assign condition rating
    if normalized_score <= 10:  # If score is low, condition is Excellent
        return "Excellent"
    elif normalized_score <= 16:  # If score is moderately low, condition is Good
        return "Good"
    elif normalized_score <= 20:  # If score is moderate, condition is Fair
        return "Fair"
    elif normalized_score <= 258:  # If score is moderately high, condition is Poor
        return "Poor"
    else:  # If score is high, condition is Very Poor
        return "Very Poor"


Request Function

In [5]:
def request_function(url):
    req = requests.get(url).text
    soup = bs(req, features="html.parser")
    return soup


Image Scrapping function

In [6]:
## Function to get images
def image_scraper(url):
    s = request_function(url)
    image_tags = s.find_all('img', class_='lazy-used-car-slider')  
    image_urls = []
    for image_tag in image_tags:
        data_src = image_tag.get('data-original')
        if data_src.endswith('webp'):
                image_urls.append(data_src)
    return image_urls

Function to Download Image

In [7]:
## function to download images
def download_img(url):
    img_data = requests.get(url).content
    # Save the image to the specified folder
    img_name = os.path.join('images', os.path.basename('new_imaeg.jpg'))
    with open(img_name, 'wb') as img_file:
       img_file.write(img_data)
   
      # now calling the car detection function
    detected_object_dic = car_detection_and_Cropping(img_name)

    return detected_object_dic



Main Scrapping funtion 

In [8]:
ad_name = []
ad_id = []
price = []
body_condition = []
engine_capacity = []
body_type = []
model_Year=[]
millage = []
fuel_type = []
transmission = []
city_registered = []
body_color = []
ad_links = []
ad_Date  = []

In [9]:
list_of_famous_cars = ['mehran','sportage','wagon-R','civic','elantra']#'sonata','corolla','swift',
l2 = ['alto','cultus','fortuner','vitz','bolan','city','alsvin','mark-x']#'tucson',




In [10]:
for car in list_of_famous_cars:
    print(car)

mehran
sportage
wagon-R
civic
elantra


compiled Function is now in the following function

In [11]:
def test_scrap(list_of_famous_cars):
    print('Loading Data ',end='')
    for car in list_of_famous_cars:
        for page in range(30,40):
            print("Page -->",page)
             
             
             
            url = ' https://www.pakwheels.com/used-cars/search/-/?q='+car+'&page='+str(page)
            s = request_function(url)
            print(url)
            ads_link =  ['https://www.pakwheels.com'+a.get('href') for a in s.find_all('a',class_='car-name ad-detail-path')]
            
            for i in ads_link:
                ad_links.append(i)

                print('ad link',i)

                s2 = request_function(i)

                id = s2.find('li', string='Ad Ref #').find_next('li').get_text() if s2.find('li', string='Ad Ref #').find_next('li').get_text() else 'N/A'
                ad_id.append(id)
                # now images
                local_img_urls = image_scraper(i)

                compiled_dic = {}
                for img_url in local_img_urls:
                    data_dictionary = download_img(img_url)
                    # here
                    for key in data_dictionary.keys():
                      if key in compiled_dic:
                        compiled_dic[key] += data_dictionary[key]
                      else:
                          compiled_dic[key] = data_dictionary[key]
                
                ad_name.append(s2.find('div',class_='well').find_next('h1').text)

                body_condition.append(estimate_condition(compiled_dic))

                model_Year.append(s2.find('span', class_ = 'engine-icon year').find_next().text.strip()
                                        if s2.find( 'span', class_='engine-icon year') else 'N/A')

                millage.append(s2.find('span', class_='engine-icon millage').find_next().text.strip())

                fuel_type.append(s2.find('span', class_='engine-icon type').find_next().text.strip())

                transmission.append(s2.find('span', class_='engine-icon transmission').find_next().text.strip())


                city_registered.append(s2.find('li', string='Registered In').find_next('li').get_text(strip=True))

                body_color.append(s2.find('li', string='Color').find_next('li').get_text(strip=True))

                engine_capacity.append(s2.find('li', string='Engine Capacity').find_next('li').get_text(strip=True)
                                            if s2.find('li',string='Engine Capacity') else s2.find(
                            'li', string='Battery Capacity').find_next('li').get_text)

                body_type.append(s2.find('li', string='Body Type').find_next('li').find('a').get_text()
                                        if s2.find('li', string='Body Type').find_next('li').find('a') else 'N/A')

                ad_Date.append(s2.find('li',string = 'Last Updated:').find_next('li').get_text()
                                              if s2.find('li', string='Last Updated:').find_next('li') else 'N/A')


                price.append(s2.find('strong',class_ = 'generic-green').text)
        
        print('Converting to DataFrame')
        df = pd.DataFrame({'AD_Name':ad_name,'AD_id':ad_id,'Engine Capacity':engine_capacity,'Category':body_type,'Body Consition':body_condition,
                       "model Year":model_Year,'Milage':millage,'Fuel Type':fuel_type,'Transmission':transmission,'Color':body_color,
                       'City Registered':city_registered,'Price':price,'Ad_link':ad_links,'ad_Date':ad_Date})

        df.to_csv("4rdTime/"+car+"_FYP DataSet30-to50.csv")

        print('Saving to CSV with Name --- > '+car)
        ad_name.clear()
        ad_id.clear()
        price.clear()
        body_condition.clear()
        engine_capacity.clear()
        body_type.clear()
        model_Year.clear()
        millage.clear()
        fuel_type.clear()
        transmission.clear()
        city_registered.clear()
        body_color.clear()
        ad_links.clear()
        ad_Date.clear()        
        

https://www.pakwheels.com/used-cars/search/-/?q=wagon-R&page=4
https://www.pakwheels.com/used-cars/search/-/?q=wagon-R&page=20

In [12]:
test_scrap(l2)

Loading Data Page --> 30
 https://www.pakwheels.com/used-cars/search/-/?q=alto&page=30
ad link https://www.pakwheels.com/used-cars/daihatsu-mira-2012-for-sale-in-islamabad-8423261
Detedtion list {'scratch': 4, 'dent': 1}
ad link https://www.pakwheels.com/used-cars/suzuki-alto-2020-for-sale-in-islamabad-8389085
Detedtion list {}
ad link https://www.pakwheels.com/used-cars/suzuki-alto-2007-for-sale-in-taxila-8492107
Detedtion list {'dent': 1, 'scratch': 3}
ad link https://www.pakwheels.com/used-cars/daihatsu-mira-2007-for-sale-in-rawalpindi-8492104
Detedtion list {'dent': 2, 'scratch': 1, 'rust': 1}
ad link https://www.pakwheels.com/used-cars/suzuki-alto-2022-for-sale-in-lahore-8492040
Detedtion list {'rust': 2, 'scratch': 5, 'crack': 1}
ad link https://www.pakwheels.com/used-cars/suzuki-alto-2023-for-sale-in-lahore-8491034
Detedtion list {}
ad link https://www.pakwheels.com/used-cars/suzuki-alto-2022-for-sale-in-rahim-yar-khan-8378656
Detedtion list {'rust': 3, 'scratch': 6}
ad link htt